In [1]:
import cv2
import pandas as pd
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
from ipywidgets import interact, Layout
import cvzone
import time
import threading
import multiprocessing as mp
from multiprocessing.shared_memory import SharedMemory
from importlib import reload
import t1, t2, t3
import pickle

In [ ]:
reload(t1)
from t1 import preprocess, contours, reorder, get_warp

font = cv2.FONT_HERSHEY_SIMPLEX
width_img = 1280
height_img = 960

with open('data/df.pkl', 'rb') as f:
    df = pickle.load(f)

ID_name = mp.Array('c', b'')
# variable.value = b'ddsds'

train = mp.Value('i', 0)
prediction = mp.Value('i', 0)

tmp = mp.Value('i', 0)
tmp2 = mp.Value('i', 0)
contours_size = mp.Value('i', 0)
rectangles_sum = mp.Value('i', 0)

frame_memory = SharedMemory(name='FrameMemory', create=True, size=width_img*height_img*3)
contours_memory = SharedMemory(name='ContoursMemory', create=True, size=4096)
rectangles_memory = SharedMemory(name='RectanglesMemory', create=True, size=100_000)

p1 = mp.Process(target=t2.ocr_detect, args=(tmp, tmp2, width_img, height_img, rectangles_sum, contours_size,), daemon=True)
p1.start()

p2 = mp.Process(target=t3.model_detect, args=(rectangles_sum, prediction, train, tmp2), daemon=True)
p2.start()

stop_button = widgets.ToggleButton(description='Stop', disabled=False)
train_button = widgets.ToggleButton(description='Train', disabled=False)
output = widgets.HTML(value=' ', description='Train:', disabled=False)
text = widgets.Text(value='', placeholder='Type ID name', description='ID_name:', disabled=False)
grid = widgets.GridBox([stop_button, output, train_button, text], layout=widgets.Layout(grid_template_columns='repeat(4, 200px)'))

# ID_card TRAIN/CHECK
def text_detection(frame, img, contours_data, df): 
    tensor = np.empty((1,4))
    for contour in contours_data[:,:]:
        x, y, w, h = contour[0], contour[2], contour[1], contour[3]
        cv2.rectangle(img, (x, y), (w, h), (255, 255, 0), 6)

    if rectangles_sum.value == 0 and train.value == 0:
        st = 'Detecting...'
        color = (0, 0, 255)
    elif train.value == 1:
        st = 'Learning...'
        color = (255, 0, 0)
    else:   
        color = (0, 255, 0)
        st = df['Name'][df['ID'] == prediction.value].iloc[0]

    cv2.putText(frame, st, (7, 70), font, 3, color, 7, cv2.LINE_AA)

def close(cap):
    cap.release()
    frame_memory.close()
    frame_memory.unlink()
    contours_memory.close()
    contours_memory.unlink()
    rectangles_memory.close()
    rectangles_memory.unlink()
    p1.terminate()
    p2.terminate()

def view(grid, df):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    frame_data = np.ndarray((height_img,width_img,3), dtype=np.uint8, buffer=frame_memory.buf)
    contours_data = np.ndarray((1,4), dtype=np.int32, buffer=contours_memory.buf)

    while True:
        _, frame = cap.read()

        frame_masked = preprocess(frame)
        frame_contours = contours(frame_masked, frame)

        # output.value = str(train.value)
        output.value = str(len(ID_name.value))

        if frame_contours.size != 0:
            img_warped = get_warp(frame, frame_contours)
            contours_size.value = 1
            if tmp.value == 1:
                frame_data[:,:,:] = img_warped[:,:,:]
                contours_data = np.ndarray((rectangles_sum.value,4), dtype=np.int32, buffer=contours_memory.buf)
                tmp.value = 0
            frame_detect = text_detection(frame, img_warped, contours_data, df)
            
        else:
            img_warped = frame.copy()
            rectangles_sum.value = 0
            frame_data[:,:,:] = img_warped[0,0,0]

        img_stack = cvzone.stackImages([
            frame,
            img_warped,
        ], 2, 1)

        _, frame = cv2.imencode('.jpeg', img_stack) 
        display_handle.update(Image(data=frame.tobytes()))
        
        
        if stop_button.value==True:
            close(cap)
            display_handle.update(None)

        if train_button.value==True:
            train.value = 1
            train_button.value = False

        if train.value == 2:
            with open('data/df.pkl', 'rb') as f:
                df = pickle.load(f)
            train.value = 0
            


display(grid)
thread = threading.Thread(target=view, args=(grid, df))
thread.start()

In [ ]:
p1.terminate()
p2.terminate()
frame_memory.close()
frame_memory.unlink()
contours_memory.close()
contours_memory.unlink()
rectangles_memory.close()
rectangles_memory.unlink()
time.sleep(0.1)
p1.is_alive()

In [3]:
print(p1.is_alive(), p2.is_alive(), thread.is_alive())

False False False


In [8]:
df = pd.DataFrame({'ID': [0, 1], 'Name': ['Karta Polaka (FRONT)', 'Karta Polaka (BACK)']})
df

,ID,Name
0,0,Karta Polaka (FRONT)
1,1,Karta Polaka (BACK)


In [10]:
with open('data/df.pkl', 'rb') as f:
    df = pickle.load(f)

In [29]:
df

,ID,Name
0,0,Karta Polaka (FRONT)
1,1,Karta Polaka (BACK)


In [8]:
out = widgets.Output()


text = widgets.Text(value='', placeholder='Type ID name', description='ID_name:', disabled=False)
display(text)

Text(value='', description='ID_name:', placeholder='Type ID name')